#### 1) Método de algum modelo de otimização

In [6]:
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpMaximize

def programacao_producao(ordens, tempos_setup):
    # Cria o problema de minimização
    prob = LpProblem("ProgramacaoProducao", LpMinimize)
    
    # Variáveis de decisão binárias indicando se a ordem i é atribuída à máquina j
    x = {(i, j): LpVariable(f'x_{i}_{j}', cat='Binary') for i in range(len(ordens)) for j in range(2)}
    
    # Função objetivo: minimizar o tempo total de processamento (e setup)
    #prob += lpSum(ordens[i] * x[(i, j)] for i in range(len(ordens)) for j in range(2))
    prob += lpSum((ordens[i] + tempos_setup[j]) * x[(i, j)] for i in range(len(ordens)) for j in range(2))
    
    # Restrições: cada ordem deve ser atribuída a exatamente uma máquina
    for i in range(len(ordens)):
        prob += lpSum(x[(i, j)] for j in range(2)) == 1
    
    # Restrições: cada máquina deve estar ocupada
    for j in range(2):
        #prob += lpSum(ordens[i] * x[(i, j)] for i in range(len(ordens))) >= 1
        prob += lpSum((ordens[i] + tempos_setup[j]) * x[(i, j)] for i in range(len(ordens))) >= 1
    
    # Resolve o problema
    prob.solve()
    
    # Obtém os resultados
    maquina1 = [ordens[i]+tempos_setup[0] for i in range(len(ordens)) if x[(i, 0)].value() == 1]
    maquina2 = [ordens[i]+tempos_setup[1] for i in range(len(ordens)) if x[(i, 1)].value() == 1]
    
    # Calcula o tempo total de processamento
    tempo_total = max(sum(maquina1), sum(maquina2))
    #tempo_total = max(sum(maquina1 + tempos_setup[0], sum(maquina2) + tempos_setup[1])
    
    # Retorna as programações e o tempo total
    return maquina1, maquina2, tempo_total

# Exemplo de ordens de operação com tempos de processamento em cada máquina
ordens = [12, 10, 8, 4]

# Exemplo de tempos de setup para cada máquina
tempos_setup = [2, 3]

# Chamada da função para realizar a programação de produção
maquina1, maquina2, tempo_total = programacao_producao(ordens, tempos_setup)


# Exibe os resultados
print("Máquina 1:", maquina1)
print("Máquina 2:", maquina2)
print("Tempo total de processamento:", tempo_total)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/wilmer/env-julian/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/690b3e20c5664636a091eea590c96b1c-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/690b3e20c5664636a091eea590c96b1c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 52 RHS
At line 59 BOUNDS
At line 68 ENDATA
Problem MODEL has 6 rows, 8 columns and 16 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 42.0667 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 0 rows, 0 columns (0 

#### 2) Método heurístico de solução

In [3]:
def programacao_producao(ordens, ts):
    # Inicializa as máquinas
    maquina1 = []
    maquina2 = []
    
    # Loop através das ordens de operação
    for ordem in ordens:
        
        # Atribui a ordem à máquina com menor tempo total
        if sum(maquina1) <= sum(maquina2):
            maquina1.append(ordem + ts[0])
        else:
            maquina2.append(ordem + ts[1])
    
    # Calcula o tempo total de processamento
    tempo_total = max(sum(maquina1), sum(maquina2))
    
    # Retorna as programações e o tempo total
    return maquina1, maquina2, tempo_total

# Exemplo de ordens de operação com tempos de processamento em cada máquina
ordens = [12, 10, 8, 4]
# tempo de setups das máquinas
ts = [2, 3] 

# Chamada da função para realizar a programação de produção
maquina1, maquina2, tempo_total = programacao_producao(ordens, ts)

# Exibe os resultados
print("Máquina 1:", maquina1)
print("Máquina 2:", maquina2)
print("Tempo total de processamento:", tempo_total)


Máquina 1: [14, 6]
Máquina 2: [13, 11]
Tempo total de processamento: 24
